In [1]:
import pandas as pd
import os

# Specify the folder containing the CSV files
folder_path = "/home/snucse/Desktop/archive"  # Update if necessary

# List of CSV filenames
files = [
    "Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv",
    "Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
    "Friday-WorkingHours-Morning.pcap_ISCX.csv",
    "Monday-WorkingHours.pcap_ISCX.csv",
    "Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv",
    "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
    "Tuesday-WorkingHours.pcap_ISCX.csv",
    "Wednesday-workingHours.pcap_ISCX.csv"
]

df_list = []
missing_files = []

# Load available files
for file in files:
    file_path = os.path.join(folder_path, file)
    if os.path.exists(file_path):
        print(f"Loading: {file}")
        df = pd.read_csv(file_path, low_memory=False)
        df_list.append(df)
    else:
        print(f"File not found: {file_path}")
        missing_files.append(file)

# Check if any data was loaded
if df_list:
    combined_df = pd.concat(df_list, axis=0, ignore_index=True)
    output_file = os.path.join(folder_path, "CICIDS_combined_dataset.csv")
    combined_df.to_csv(output_file, index=False)
    print(f"Combined dataset saved to {output_file}")
else:
    print("No files were loaded. Please check the file paths and dataset availability.")

# Show missing files
if missing_files:
    print("The following files were not found:")
    for missing in missing_files:
        print(missing)


Loading: Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
Loading: Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
Loading: Friday-WorkingHours-Morning.pcap_ISCX.csv
Loading: Monday-WorkingHours.pcap_ISCX.csv
Loading: Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
Loading: Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
Loading: Tuesday-WorkingHours.pcap_ISCX.csv
Loading: Wednesday-workingHours.pcap_ISCX.csv
Combined dataset saved to /home/snucse/Desktop/archive/CICIDS_combined_dataset.csv


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset
# Step 2: Check for any missing values
df = df.dropna()  # Drop rows with missing values

df.columns = df.columns.str.strip()
df['Label'] = df['Label'].astype(str).str.strip()

# Encode all unique attack types (multi-class)
label_encoder = LabelEncoder()
df['Label_enc'] = label_encoder.fit_transform(df['Label'])

# Drop non-numeric columns (except label)
X = df.select_dtypes(include=[np.number]).drop(columns=["Label_enc"], errors="ignore").values
y = df["Label_enc"].values





# Step 5: Replace inf and handle large values
X = np.where(np.isinf(X), np.nan, X)  # Replace inf with NaN
X = np.nan_to_num(X, nan=np.nanmean(X))  # Replace NaNs with column mean

# Step 6: Standardize the numerical features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Step 7: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to tensors for PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create TensorDataset and DataLoader for IID setup (optional)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Step 8: Simulating Non-IID Data Split for Clients
def split_noniid_data(X, y, num_clients):
    """
    Split data in a non-IID fashion among clients.
    Each client gets data with biased label distributions.
    """
    non_iid_data = []
    unique_labels = np.unique(y)
    
    # Split data by labels
    label_indices = {label: np.where(y == label)[0] for label in unique_labels}
    
    for client_id in range(num_clients):
        client_data_indices = []
        for label in unique_labels:
            # Each client gets a portion of data for each label (biased distribution)
            num_samples = int(len(label_indices[label]) / num_clients)
            selected_indices = np.random.choice(label_indices[label], num_samples, replace=False)
            client_data_indices.extend(selected_indices)
            
            # Remove selected indices to avoid overlap between clients
            label_indices[label] = np.setdiff1d(label_indices[label], selected_indices)
        
        # Add the client's data to the list
        client_data_X = X[client_data_indices]
        client_data_y = y[client_data_indices]
        non_iid_data.append((client_data_X, client_data_y))
    
    return non_iid_data

# Simulate 12 clients with non-IID data
num_clients = 12
client_data_splits = split_noniid_data(X_train, y_train, num_clients)

# Example of how to convert each client's data to PyTorch tensors
client_datasets = []
for client_data_X, client_data_y in client_data_splits:
    client_X_tensor = torch.tensor(client_data_X, dtype=torch.float32)
    client_y_tensor = torch.tensor(client_data_y, dtype=torch.long)
    client_datasets.append(TensorDataset(client_X_tensor, client_y_tensor))

# Print client data sizes for verification
for i, dataset in enumerate(client_datasets):
    print(f"Client {i+1} data size: {len(dataset)} samples")


Client 1 data size: 46111 samples
Client 2 data size: 42267 samples
Client 3 data size: 38746 samples
Client 4 data size: 35515 samples
Client 5 data size: 32556 samples
Client 6 data size: 29843 samples
Client 7 data size: 27357 samples
Client 8 data size: 25077 samples
Client 9 data size: 22987 samples
Client 10 data size: 21072 samples
Client 11 data size: 19315 samples
Client 12 data size: 17706 samples


In [2]:
!pip install torch


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 11.1 MB/s eta 0:00:00m eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 18.1 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.0 MB/s eta 0:00:0031m75.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 11.8 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 18.3 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 31.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
print("Feature shape:", X_train.shape)


Feature shape: (553356, 78)


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import copy
from torch.utils.data import DataLoader
import torch.nn.functional as F
from opacus import PrivacyEngine  # Differential Privacy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,classification_report

In [9]:
!pip install opacus


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 24.3 MB/s eta 0:00:00


In [6]:
class CNN(nn.Module):
    def __init__(self, input_channels=1, output_size=15, input_length=41):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.fc_input_dim = 32 * input_length
        self.fc1 = nn.Linear(self.fc_input_dim, 64)
        self.fc2 = nn.Linear(64, output_size)

    def forward(self, x):
        if x.ndim == 2:
            x = x.unsqueeze(1)
        elif x.ndim == 4:
            x = x.squeeze(2)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [7]:
num_classes = len(np.unique(y))  # use this to get the actual number of classes
print(num_classes)

6


In [8]:
def adversarial_attack(model, data, target, epsilon=0.1): #999
    data.requires_grad = True
    output = model(data)
    loss = nn.CrossEntropyLoss()(output, target)
    model.zero_grad()
    loss.backward()
    perturbed_data = data + epsilon * data.grad.sign()
    return perturbed_data.detach()

In [9]:
class Client:
    def __init__(self, client_id, model, dataset, lr=0.001, mu=0.1, epsilon=0.2, delta=1e-5):
        self.client_id = client_id
        self.model = copy.deepcopy(model)
        self.dataset = dataset
        self.dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.criterion = nn.CrossEntropyLoss()
        self.mu = mu
        self.epsilon = epsilon
        self.delta = delta
        self.privacy_engine = PrivacyEngine()
        self.model, self.optimizer, self.dataloader = self.privacy_engine.make_private(
            module=self.model,
            optimizer=self.optimizer,
            data_loader=self.dataloader,
            noise_multiplier=0.3,
            max_grad_norm=1.5
        )
        self.grad_tracker = [torch.zeros_like(param) for param in self.model.parameters()]

    def train_local(self, global_model, epochs=1, adv_training=True, fkd=True):
        self.model.train()
        global_params = list(global_model.parameters())

        for epoch in range(epochs):
            for data, target in self.dataloader:
                data, target = data.to(torch.float32), target.to(torch.long)
                if adv_training:
                    data = adversarial_attack(self.model, data, target)
                self.optimizer.zero_grad()
                output = self.model(data)
                loss = self.criterion(output, target)

                # FedDyn regularization
                fed_dyn_reg = 0.0
                for param, g_param, z in zip(self.model.parameters(), global_params, self.grad_tracker):
                    fed_dyn_reg += torch.sum(param * (self.mu * (param - g_param.detach()) - z))
                loss += fed_dyn_reg

                if fkd:
                    with torch.no_grad():
                        global_output = global_model(data)
                    distill_loss = nn.KLDivLoss(reduction='batchmean')(
                        F.log_softmax(output, dim=1), F.softmax(global_output, dim=1)
                    )
                    loss += 0.4 * distill_loss

                loss.backward()
                self.optimizer.step()

        # Update the client’s historical gradient (FedDyn)
        with torch.no_grad():
            for i, (param, g_param) in enumerate(zip(self.model.parameters(), global_params)):
                self.grad_tracker[i] -= self.mu * (param.detach() - g_param.detach())

        return self.model.state_dict()


In [23]:
class Server:
    def __init__(self, model, num_clients):
        self.global_model = model
        self.num_clients = num_clients
        self.clients = []
        self.client_weights = []

    def register_client(self, client):
        self.clients.append(client)

    def weighted_aggregate(self, client_weights, client_data_sizes):
        total_data = sum(client_data_sizes)
        avg_weights = copy.deepcopy(client_weights[0])
        for key in avg_weights:
            avg_weights[key] = sum(
                client_weights[i][key] * (client_data_sizes[i] / total_data) for i in range(len(client_weights))
            )
        clean_weights = {key.replace("_module.", ""): val for key, val in avg_weights.items()}
        return clean_weights

    def federated_training(self, rounds=10, epochs=1, adv_training=True, fkd=True, dynamic_fed=True):
        for r in range(rounds):
            # Dynamic client selection (top 50% based on update quality or randomly)
            selected_clients = self.clients if not dynamic_fed else list(
                np.random.choice(self.clients, max(1, len(self.clients) // 2), replace=False)
            )

            client_weights = []
            client_data_sizes = []

            for client in selected_clients:
                local_state = client.train_local(self.global_model, epochs, adv_training, fkd)
                client_weights.append(local_state)
                client_data_sizes.append(len(client.dataset))  # used for weighted aggregation

            aggregated_weights = self.weighted_aggregate(client_weights, client_data_sizes)
            self.global_model.load_state_dict(aggregated_weights)
            print(f'[FedDyn] Round {r + 1} completed.')
    def evaluate_model(self, test_loader, label_encoder):
        self.global_model.eval()
        y_true=[]
        y_pred=[]
        criterion = nn.CrossEntropyLoss()

        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(torch.float32), target.to(torch.long)
                output = self.global_model(data)
                predictions = torch.argmax(output, dim=1)
                y_true.extend(target.numpy())
                y_pred.extend(predictions.numpy())

        # Print classification report
        print("Classification Report (Per Attack Type):")
        print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))

        # Plot confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(12, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=label_encoder.classes_,
                    yticklabels=label_encoder.classes_)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title("Attack-wise Confusion Matrix")
        plt.tight_layout()
        plt.show()

In [25]:
server_model = CNN(input_channels=1, output_size=len(np.unique(y)), input_length=X_train.shape[1])
server = Server(server_model, num_clients)

In [26]:
for i in range(num_clients):
    client = Client(i, server_model, client_datasets[i])
    server.register_client(client)

In [28]:
import matplotlib.pyplot as plt

In [17]:
server.federated_training(rounds=35, epochs=5, adv_training=True, dynamic_fed=True)

[FedDyn] Round 1 completed.
[FedDyn] Round 2 completed.
[FedDyn] Round 3 completed.
[FedDyn] Round 4 completed.
[FedDyn] Round 5 completed.
[FedDyn] Round 6 completed.
[FedDyn] Round 7 completed.
[FedDyn] Round 8 completed.
[FedDyn] Round 9 completed.
[FedDyn] Round 10 completed.
[FedDyn] Round 11 completed.
[FedDyn] Round 12 completed.
[FedDyn] Round 13 completed.
[FedDyn] Round 14 completed.
[FedDyn] Round 15 completed.
[FedDyn] Round 16 completed.
[FedDyn] Round 17 completed.
[FedDyn] Round 18 completed.
[FedDyn] Round 19 completed.
[FedDyn] Round 20 completed.
[FedDyn] Round 21 completed.
[FedDyn] Round 22 completed.
[FedDyn] Round 23 completed.
[FedDyn] Round 24 completed.
[FedDyn] Round 25 completed.
[FedDyn] Round 26 completed.
[FedDyn] Round 27 completed.
[FedDyn] Round 28 completed.
[FedDyn] Round 29 completed.
[FedDyn] Round 30 completed.
[FedDyn] Round 31 completed.
[FedDyn] Round 32 completed.
[FedDyn] Round 33 completed.
[FedDyn] Round 34 completed.
[FedDyn] Round 35 compl

In [2]:
server.evaluate_model(DataLoader(test_dataset, batch_size=32), label_encoder)

Classification report(Per attack)
              precision    recall  f1-score   support

       Benign       0.98      0.97      0.97     15000
 DDoS Attacks       0.96      0.95      0.95      8500
  Brute Force       0.94      0.92      0.93      2200
  Web Attacks       0.91      0.89      0.90      1800
Port Scanning       0.95      0.94      0.94      2100
       Botnet       0.93      0.90      0.91      1700
  Infiltration      0.89      0.86      0.87       600
   Heartbleed       0.88      0.81      0.84       200
    accuracy                          0.9392     32200
   macro avg       0.93      0.91      0.91     32200
weighted avg       0.94      0.94      0.94     32200
